In [1]:
import pandas as pd
from pathlib import Path
import seaborn as sns
from scipy.stats import pearsonr
df = pd.read_csv("speeches.csv.tar.gz")

/tmp/ipykernel_4042947/2820744744.py:5: DtypeWarning: Columns (11,12,13,14,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("speeches.csv.tar.gz")


In [2]:
# Filtering:
# Keep only MPs
c1 = (df.Speaker_MP == "MP")

# Keep only speeches where speaker is either Opposition or Coalition:
c2 = df.Party_status.isin(["Opposition", "Coalition"])
df = df[c1&c2].reset_index(drop=True)


grouped = df.groupby(['Speaker_name', 'Party_status']).size().reset_index(name='count').pivot(index='Speaker_name', columns='Party_status', values='count').fillna(0)
filtered_speakers = grouped[(grouped['Coalition'] >= 10) & (grouped['Opposition'] >= 10)].index

ndf = df[df['Speaker_name'].isin(filtered_speakers)]
results = dict()
for country in sorted(df.country.unique().tolist()):
    subset = ndf[ndf.country == country]
    pivoted = subset.groupby(["Speaker_name", "Party_status"]).logits_pondered.mean().reset_index().pivot(
        index='Speaker_name', columns='Party_status', values='logits_pondered'
    )
    try:
        pivoted['Difference'] = pivoted['Coalition'] - pivoted['Opposition']
        current_mean = pivoted.Difference.mean()
        current_support = pivoted.shape[0]
    except KeyError:
        current_mean = None
        current_support = 0
    results[country] = {"Difference": current_mean,
                        "Support": current_support}
turning = pd.DataFrame(results).T


In [3]:
turning

,Difference,Support
AT,0.709242,56.0
BA,-0.065664,15.0
BE,0.251016,70.0
BG,0.451272,43.0
CZ,0.567185,84.0
DK,0.468849,104.0
EE,NaN,0.0
ES,0.494550,74.0
FI,0.534082,93.0
FR,0.209120,10.0
